# Introduction
The purpose of this notebook is to compare the results of running Olivo's GRC tools on the "7979" bam files which form a superset of Pf5.1, and the more recent Pf6.0 release bams. Perhaps the most significant difference in these two runs is using bwa aln (5.1) vs bwa mem (6.0). I would need to look at full pipeline of 5.1 to understand other differences, e.g. was BQSR used? What about snp-o-matic?

See 20161111_run_Olivo_GRC.ipynb for details of running Olivo's GRC code on the Pf6.0 release. Olivo emailed me the results on the 7979 samples (samplesMeta5x-V1.0.xlsx) 10/11/2016 14:44

In [1]:
%run _standard_imports.ipynb

python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.19.0
numexpr 2.6.1
pysam 0.9.1.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [46]:
output_dir = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161114_compare_7979_vs_Pf6_GRC"
!mkdir -p {output_dir}
olivo_7979_results_fn = "%s/samplesMeta5x-V1.0.xlsx" % output_dir
richard_6_0_results_fn = "/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161111_run_Olivo_GRC/grc/AllCallsBySample.tab"

bwa_aln_fofn = '/nfs/team112_internal/rp7/data/Pf/hrp/metadata/hrp_manifest_20160621.txt'
bwa_mem_fofn = '/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161111_run_Olivo_GRC/pf_60_mergelanes.txt'

In [47]:
!wc -l {bwa_aln_fofn}

7980 /nfs/team112_internal/rp7/data/Pf/hrp/metadata/hrp_manifest_20160621.txt


In [48]:
!wc -l {bwa_mem_fofn}

8252 /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161111_run_Olivo_GRC/pf_60_mergelanes.txt


In [50]:
# 3D7 reads for Sorina (nothing to do with GRC!) - see email from Jacob 14/11/2016 15:08
!grep PG0567 {bwa_mem_fofn}

/lustre/scratch116/malaria/pfalciparum/output/6/0/9/9/40705/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PG0567-C


In [6]:
tbl_bwa_aln = (
    etl
    .fromxlsx(olivo_7979_results_fn)
)
print(len(tbl_bwa_aln.data()))
tbl_bwa_aln

7979


Num,Sample,OxfordSrcCode,OxfordDonorCode,AlfrescoStudyCode,Study,Type,NotAmplified,InV5,InV4,SubContinentCode,SubContinentName,RegionCode,CountryCode,CountryName,Location,Year,Date,HasDuplicate,DiscardAsDuplicate,ManuallyExcluded,Usable,Typability-100kSnps,Typable75,Typable80,Typable95,MedianCoverage-100kSnps,MeanCoverage-100kSnps,Stdev-100kSnps,CoefVarCoverage-100kSnps,Bases5Xcoverage,MeanReadCoverage,Sequencing reads,BamFile,Species,Fws,Clonal,ArtHalfLife,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],arps10-128[D],mdr1-CN-refgenecov,mdr1-CN-covAnalysis,mdr1-CN-HMM,pm23-CN-HMM,pm23-join-amplified,K13Common,K13CommonAllele,WAF-PC1,WAF-PC1-Class,aat-S258L
214,PA0236-C,ZUC006,ZUC006,1026-PF-GN-CONWAY,PF26,dna,True,True,True,WAF,West Africa,GN,GN,Guinea,Nzerekore,-,-,False,False,False,"=IF(T2,""Duplicate"",IF(NOT(X2),""Lt75PcCoverage"",IF(N2=""Lab"",""Lab"",IF(U2<>FALSE,""ManuallyExcluded"",TRUE))))",0,False,False,False,1,0.986779,0.216146,0.219041953669464,32,1.04,ERS042064,/lustre/scratch109/malaria/pfalciparum/output/c/5/a/5/42934/1_bam_merge/pe.1.bam,-,-,True,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,MI,-,-,-,-
240,PA0265-C,ZRL199,ZRL199,1026-PF-GN-CONWAY,PF26,dna,True,True,True,WAF,West Africa,GN,GN,Guinea,Nzerekore,-,-,False,False,False,"=IF(T3,""Duplicate"",IF(NOT(X3),""Lt75PcCoverage"",IF(N3=""Lab"",""Lab"",IF(U3<>FALSE,""ManuallyExcluded"",TRUE))))",0,False,False,False,1,1.000267,0.196664,0.19661150472823757,172,1.05,ERS042008,/lustre/scratch109/malaria/pfalciparum/output/2/2/a/1/42963/1_bam_merge/pe.1.bam,-,-,True,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,MI,-,-,-,-
328,PC0109-C,119,119,1027-PF-KE-BULL,PF27,dna,True,True,True,EAF,East Africa,KE,KE,Kenya,Kilifi,-,-,False,False,False,"=IF(T4,""Duplicate"",IF(NOT(X4),""Lt75PcCoverage"",IF(N4=""Lab"",""Lab"",IF(U4<>FALSE,""ManuallyExcluded"",TRUE))))",0,False,False,False,1,0.997015,0.104629,0.10494225262408288,0,1.02,ERS132630,/lustre/scratch109/malaria/pfalciparum/output/6/6/f/a/43040/1_bam_merge/pe.1.bam,-,-,True,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,MI,-,-,-,-
339,PC0141-C,KC-150,KC-150,1111-PF-LAB3D7-BULL,1111-PF-LAB3D7-BULL,dna,True,True,True,EAF,East Africa,KE,KE,Kenya,-,-,-,False,False,False,"=IF(T5,""Duplicate"",IF(NOT(X5),""Lt75PcCoverage"",IF(N5=""Lab"",""Lab"",IF(U5<>FALSE,""ManuallyExcluded"",TRUE))))",0,False,False,False,1,1.033254,0.221906,0.21476423028606714,-,-,ERS384317,/lustre/scratch109/malaria/pfalciparum/output/1/8/f/f/203086/1_bam_merge/pe.1.bam,-,-,True,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,MI,-,-,-,-
347,PC0149-C,KC-127,KC-127,1111-PF-LAB3D7-BULL,1111-PF-LAB3D7-BULL,dna,True,True,True,EAF,East Africa,KE,KE,Kenya,-,-,-,False,False,False,"=IF(T6,""Duplicate"",IF(NOT(X6),""Lt75PcCoverage"",IF(N6=""Lab"",""Lab"",IF(U6<>FALSE,""ManuallyExcluded"",TRUE))))",0,False,False,False,1,1.025199,0.196882,0.19204271560935976,-,-,ERS384325,/lustre/scratch109/malaria/pfalciparum/output/3/2/1/e/203094/1_bam_merge/pe.1.bam,-,-,True,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,MI,-,-,-,-


In [15]:
tbl_bwa_mem = (
    etl
    .fromtsv(richard_6_0_results_fn)
    .rename('mdr2_484[P]', 'mdr2_484[T]')
    .rename('fd_193[P]', 'fd_193[D]')
)
print(len(tbl_bwa_mem.data()))
tbl_bwa_mem

8251


Num,Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
1,PL0006-C,CVMNK,D,H,A,L,I,A,Q,N,T,"T,A","I,L",R,N,C,"S,N",I,S,"G,A",K,A,A,N,Y,D,V,T,D,E
2,WS0081-C,CVMNK,D,H,A,L,I,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
3,QJ0018-C,CVIET,D,H,A,L,I,S,E,N,T,T,I,I,I,R,N,I,S,G,K,A,A,Y,F,D,V,T,D,E
4,PH0458-CW,CVIET,D,H,A,L,I,S,E,S,T,T,T,I,I,R,N,L,S,G,N,G,A,N,F,D,M,I,Y,G
5,PD0986-C,CVIET,D,H,A,L,i,s,E,s,t,T,T,I,I,R,N,L,A,G,E,A,A,N,Y,D,M,T,Y,E


In [28]:
loci = list(tbl_bwa_mem.header()[2:])
print(len(loci))

29


In [21]:
tbl_both_results = (
    tbl_bwa_aln
    .cut(['Sample'] + loci)
    .join(tbl_bwa_mem.cut(['Sample'] + loci), key='Sample')
)
print(len(tbl_both_results.data()))


7840


In [22]:
tbl_both_results

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
FP0008-C,"CVIET,CVMNK",D,H,A,L,I,S,E,N,T,T,"T,I","I,R","N,I","C,R","S,N",I,"A,S","A,G",K,A,A,N,"F,Y",D,V,T,D,E,"CVIET,CVMNK",D,H,A,L,I,S,E,N,T,T,"T,I","I,R","N,I","C,R","S,N",I,"A,S","A,G",K,A,A,N,"F,Y",D,V,T,D,E
FP0009-C,CVIET,D,H,A,L,I,S,E,N,T,T,T,I,I,R,N,I,A,A,K,A,A,Y,F,"Y,D",V,T,D,E,CVIET,D,H,A,L,I,S,E,N,T,T,T,I,I,R,N,I,A,A,K,A,A,Y,F,"Y,D",V,T,D,E
FP0015-C,CVIET,D,H,A,L,I,S,E,N,T,T,I,I,I,R,N,I,"A,S","A,G",K,A,A,N,"F,Y",D,V,T,D,E,CVIET,D,H,A,L,I,S,E,N,T,T,I,I,I,R,N,I,"A,S","A,G",K,A,A,N,"F,Y",D,V,T,D,E
FP0016-C,CVIET,D,h,A,l,i,s,E,n,t,t,i,I,N,C,S,I,S,G,K,A,A,N,F,D,-,T,D,E,CVIET,D,h,A,l,i,s,E,n,t,t,i,I,N,C,S,I,S,G,K,A,A,N,F,D,-,T,D,E
FP0017-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,S,I,"A,F",A,K,A,"A,S",N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,S,I,"A,F",A,K,A,"A,S",N,Y,D,V,T,D,E


In [29]:
tbl_both_results.values(30).array()

array(['CVIET,CVMNK', 'CVIET', 'CVIET', ..., 'CVMNK', 'CVMNK', 'CVMNK'], 
      dtype='<U11')

In [38]:
for i in range(1, 30):
    print(i, loci[i-1])
    tbl_both_results.valuecounts(i, i+29).displayall()

1 crt_72-76[CVMNK]


1,30,count,frequency
CVIET,CVIET,3631,0.4631377551020408
CVMNK,CVMNK,2318,0.29566326530612247
-,-,346,0.04413265306122449
"CVMNK,CVIET","CVMNK,CVIET",301,0.038392857142857145
CVIDT,CVIDT,272,0.03469387755102041
"CVIET,CVMNK","CVIET,CVMNK",243,0.030994897959183674
SVMNT,SVMNT,225,0.028698979591836735
cviet,cviet,206,0.026275510204081632
cvmnk,cvmnk,65,0.008290816326530613
"CVIET,CVIDT","CVIET,CVIDT",39,0.004974489795918367


2 crt_24[D]


2,31,count,frequency
D,D,7075,0.9024234693877551
-,-,325,0.04145408163265306
d,d,300,0.03826530612244898
"D,Y","D,Y",60,0.007653061224489796
Y,Y,27,0.0034438775510204083
"Y,D","Y,D",21,0.0026785714285714286
D,d,11,0.0014030612244897958
d,D,9,0.0011479591836734695
d,-,4,0.0005102040816326531
-,D,3,0.0003826530612244898


3 crt_97[H]


...

27,56,count,frequency
T,T,5293,0.6751275510204081
I,I,1862,0.2375
-,-,236,0.03010204081632653
"I,T","I,T",145,0.018494897959183673
"T,I","T,I",127,0.016198979591836735
t,t,101,0.01288265306122449
i,i,57,0.007270408163265306
-,T,3,0.0003826530612244898
t,-,3,0.0003826530612244898
t,T,2,0.00025510204081632655


28 fd_193[D]


28,57,count,frequency
D,D,5093,0.6496173469387755
Y,Y,2012,0.2566326530612245
-,-,298,0.03801020408163265
d,d,163,0.020790816326530612
y,y,95,0.01211734693877551
"D,Y","D,Y",78,0.009948979591836734
"Y,D","Y,D",71,0.009056122448979591
d,-,7,0.0008928571428571428
D,d,5,0.0006377551020408163
y,-,4,0.0005102040816326531


29 exo_415[E]


29,58,count,frequency
E,E,6999,0.8927295918367347
G,G,337,0.04298469387755102
-,-,226,0.028826530612244897
e,e,176,0.022448979591836733
"E,G","E,G",27,0.0034438775510204083
"G,E","G,E",27,0.0034438775510204083
g,g,14,0.0017857142857142857
"E,E","E,E",7,0.0008928571428571428
e,-,5,0.0006377551020408163
E,e,4,0.0005102040816326531


In [ ]:
dhfr_108, S vs N
dhps_437, G vs A
mdr1_184, F vs Y

In [39]:
tbl_both_results.selecteq(16, 'S').selecteq(16+29, 'N')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0282-C,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,i,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0304-C,-,D,H,-,-,i,a,Q,n,t,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0312-C,cvmnk,D,h,-,-,i,-,Q,N,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0313-C,CVMNK,D,H,a,l,I,-,Q,n,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,d,H,A,l,i,a,Q,n,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E


In [40]:
tbl_both_results.selecteq(16, 'N').selecteq(16+29, 'S')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0330-C,CVMNK,D,H,A,L,-,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0332-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,-,D,H,-,-,i,a,Q,n,t,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0334-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,cvmnk,D,h,-,-,i,-,Q,N,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0335-C,cvmnk,D,H,A,L,-,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,CVMNK,D,H,a,l,i,-,Q,n,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E


In [72]:
tbl_both_results.selecteq(19, 'G').selecteq(19+29, 'A')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0282-C,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,i,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0304-C,-,D,H,-,-,i,a,Q,n,t,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0312-C,cvmnk,D,h,-,-,i,-,Q,N,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0313-C,CVMNK,D,H,a,l,I,-,Q,n,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,d,H,A,l,i,a,Q,n,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E


In [73]:
tbl_both_results.selecteq(19, 'A').selecteq(19+29, 'G')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0330-C,CVMNK,D,H,A,L,-,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0332-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,-,D,H,-,-,i,a,Q,n,t,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0334-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,cvmnk,D,h,-,-,i,-,Q,N,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0335-C,cvmnk,D,H,A,L,-,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,CVMNK,D,H,a,l,i,-,Q,n,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E


In [41]:
tbl_both_results.selecteq(24, 'F').selecteq(24+29, 'Y')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0330-C,CVMNK,D,H,A,L,-,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0332-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,-,D,H,-,-,i,a,Q,n,t,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0334-C,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,cvmnk,D,h,-,-,i,-,Q,N,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E
PG0335-C,cvmnk,D,H,A,L,-,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E,CVMNK,D,H,a,l,i,-,Q,n,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E


In [42]:
tbl_both_results.selecteq(24, 'Y').selecteq(24+29, 'F')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E],crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0282-C,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,i,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0304-C,-,D,H,-,-,i,a,Q,n,t,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0312-C,cvmnk,D,h,-,-,i,-,Q,N,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,D,H,A,L,I,A,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E
PG0313-C,CVMNK,D,H,a,l,I,-,Q,n,T,T,I,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E,CVMNK,d,H,A,l,i,a,Q,n,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E


In [44]:
tbl_both_results.selecteq(24, 'Y').selecteq(24+29, 'F').values('Sample').array()

array(['PG0282-C', 'PG0304-C', 'PG0312-C', 'PG0313-C'], 
      dtype='<U8')

In [45]:
tbl_both_results.selecteq(24, 'F').selecteq(24+29, 'Y').values('Sample').array()

array(['PG0330-C', 'PG0332-C', 'PG0334-C', 'PG0335-C'], 
      dtype='<U8')

In [75]:
!grep PG0282 {bwa_aln_fofn}

In [74]:
!grep PG0282 {bwa_mem_fofn}

/lustre/scratch116/malaria/pfalciparum/output/6/4/a/8/40437/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PG0282-C


In [79]:
!grep PG0330 {bwa_mem_fofn}

/lustre/scratch116/malaria/pfalciparum/output/e/7/0/d/40470/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PG0330-C


In [76]:
!grep PG0282 /nfs/team112_internal/production_files/Pf/5_1/pf_51_samplebam_cleaned.fofn

/lustre/scratch109/malaria/pfalciparum/output/3/4/8/c/59566/1_bam_merge/pe.1.bam	PG0282-C	1040-PF-GB-HAMILTON


In [78]:
# See methods-dev/notebooks/20160621_HRP_sample_metadata.ipynb
fofns = collections.OrderedDict()

fofns['pf_community_5_0'] = '/nfs/team112_internal/production/release_build/Pf/5_0_release_packages/pf_50_freeze_manifest_nolab_olivo.tab'
fofns['pf_community_5_1'] = '/nfs/team112_internal/production_files/Pf/5_1/pf_51_samplebam_cleaned.fofn'
fofns['pf3k_pilot_5_0_broad'] = '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/pf3k_metadata.tab'
fofns['pdna'] = '/nfs/team112_internal/production_files/Pf/PDNA/pf_pdna_new_samplebam.fofn'
fofns['conway'] = '/nfs/team112_internal/production_files/Pf/1147_Conway/pf_conway_metadata.fofn'
fofns['trac'] = '/nfs/team112_internal/rp7/data/Pf/hrp/fofns/olivo_TRAC.fofn'
fofns['fanello'] = '/nfs/team112_internal/rp7/data/Pf/hrp/fofns/olivo_fanello.fofn'

for fofn in fofns:
    print(fofn)
    !grep PG0282 {fofns[fofn]}


pf_community_5_0
pf_community_5_1
/lustre/scratch109/malaria/pfalciparum/output/3/4/8/c/59566/1_bam_merge/pe.1.bam	PG0282-C	1040-PF-GB-HAMILTON
pf3k_pilot_5_0_broad
pdna
conway
trac
fanello


In [67]:
def show_rg(sample='PG0282'):
    line = !grep {sample} {bwa_aln_fofn}
    bam_fn = line[0].split('\t')[0]
    rg = !samtools view -H {bam_fn} | grep RG
    print(rg[0])
    line = !grep {sample} {bwa_mem_fofn}
    bam_fn = line[0].split('\t')[0]
    rg = !samtools view -H {bam_fn} | grep RG
    print(rg[0])

In [68]:
for sample_id in ['PG0282-C', 'PG0304-C', 'PG0312-C', 'PG0313-C', 'PG0330-C', 'PG0332-C', 'PG0334-C', 'PG0335-C']:
    print(sample_id)
    show_rg(sample_id)

PG0282-C
@RG	ID:9146_6_16	PL:ILLUMINA	PU:9146_6_16	LB:PG0282_C_6451898	DS:PFM	SM:PG0282-C	CN:SC
@RG	ID:9146_6_15	LB:PG0282_C_6451898	PL:ILLUMINA	SM:PG0282-C	PU:9146_6_15	CN:SC	DS:1040-PF-GB-HAMILTON
PG0304-C
@RG	ID:9146_8_16	PL:ILLUMINA	PU:9146_8_16	LB:PG0304_C_6453573	DS:PFM	SM:PG0304-C	CN:SC
@RG	ID:9146_8_13	LB:PG0304_C_6453573	PL:ILLUMINA	SM:PG0304-C	PU:9146_8_13	CN:SC	DS:1040-PF-GB-HAMILTON
PG0312-C
@RG	ID:9146_8_23	PL:ILLUMINA	PU:9146_8_23	LB:PG0312_C_6453580	DS:PFM	SM:PG0312-C	CN:SC
@RG	ID:9146_8_21	LB:PG0312_C_6453580	PL:ILLUMINA	SM:PG0312-C	PU:9146_8_21	CN:SC	DS:1040-PF-GB-HAMILTON
PG0313-C
@RG	ID:9146_8_24	PL:ILLUMINA	PU:9146_8_24	LB:PG0313_C_6453581	DS:PFM	SM:PG0313-C	CN:SC
@RG	ID:9146_8_22	LB:PG0313_C_6453581	PL:ILLUMINA	SM:PG0313-C	PU:9146_8_22	CN:SC	DS:1040-PF-GB-HAMILTON
PG0330-C
@RG	ID:9146_6_13	PL:ILLUMINA	PU:9146_6_13	LB:PG0330_C_6451895	DS:PFM	SM:PG0330-C	CN:SC
@RG	ID:9146_6_16	LB:PG0330_C_6451895	PL:ILLUMINA	SM:PG0330-C	PU:9146_6_16	CN:SC	DS:1040-PF-GB-HAMILTON
PG033

In [69]:
tbl_bwa_aln.cut(['Sample'] + loci).selecteq('Sample', 'PG0282-C')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0282-C,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E


In [70]:
tbl_bwa_mem.cut(['Sample'] + loci).selecteq('Sample', 'PG0330-C')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0330-C,CVMNK,d,H,-,-,-,-,Q,n,t,t,i,R,N,C,S,I,S,G,K,A,A,N,Y,D,V,T,D,E


In [71]:
tbl_bwa_mem.cut(['Sample'] + loci).selecteq('Sample', 'PG0282-C')

Sample,crt_72-76[CVMNK],crt_24[D],crt_97[H],crt_144[A],crt_148[L],crt_194[I],crt_220[A],crt_271[Q],crt_326[N],crt_333[T],crt_342[T],crt_356[I],crt_371[R],dhfr_51[N],dhfr_59[C],dhfr_108[S],dhfr_164[I],dhps_436[S],dhps_437[G],dhps_540[K],dhps_581[A],dhps_613[A],mdr1_86[N],mdr1_184[Y],mdr1_1246[D],arps10_127[V],mdr2_484[T],fd_193[D],exo_415[E]
PG0282-C,CVMNK,D,H,A,L,i,a,Q,N,T,T,I,R,N,C,N,I,S,A,K,A,A,N,F,D,V,T,D,E


In [52]:
line = !grep PG0282 {bwa_aln_fofn}

In [ ]:
!samtools view -H /lustre/scratch109/malaria/pfalciparum/output/2/c/8/5/59596/1_bam_merge/pe.1.bam | grep RG

In [51]:
!grep PG0282 {bwa_mem_fofn}

/lustre/scratch116/malaria/pfalciparum/output/6/4/a/8/40437/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PG0282-C


In [53]:
!grep PG0330 {bwa_aln_fofn}

In [55]:
!samtools view -H /lustre/scratch109/malaria/pfalciparum/output/2/c/8/5/59596/1_bam_merge/pe.1.bam | grep RG

@RG	ID:9146_6_13	PL:ILLUMINA	PU:9146_6_13	LB:PG0330_C_6451895	DS:PFM	SM:PG0330-C	CN:SC


In [54]:
!grep PG0330 {bwa_mem_fofn}

/lustre/scratch116/malaria/pfalciparum/output/e/7/0/d/40470/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PG0330-C


# Conclusion
Manual inspection of the above tables shows that there are no major differences between the two runs. Other than for 4 pairs on clone tree samples (that looks like a potential sample mislabelling) I couldn't see any truly discordant results, just upper case to lower case or called to uncalled differences.